# 1.1 

In [1]:
import time
import os
from urllib.request import urlopen
from selenium import webdriver # allow launching browser
from selenium.webdriver.common.by import By # allow search with parameters
from selenium.webdriver.support.ui import WebDriverWait # allow waiting for page to load
from selenium.webdriver.support import expected_conditions as EC # determine whether the web page has loaded
from selenium.common.exceptions import TimeoutException # handling timeout situation
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
from datetime import date 

In [8]:
driver_option = webdriver.ChromeOptions()
driver_option.add_argument('--headless')
# driver_option.add_argument('--incognito')
chromedriver_path = '/Users/ellpeeaxe/Desktop/SMU/Y4S1/SA/chromedriver' # Change this to your own chromedriver path!

def create_webdriver():
    return webdriver.Chrome(executable_path=chromedriver_path, chrome_options=driver_option)

In [9]:
def get_posturls(username):
    
    browser = create_webdriver()
    # Open the website
    browser.get('https://www.instagram.com/'+username)

    # Get the number of posts
    post_num_element = browser.find_element_by_class_name('g47SY')
    post_num_html = post_num_element.get_attribute('innerHTML')

    # Retrieve number of posts
    soup = BeautifulSoup(post_num_html, 'html.parser')
    num_posts = int(soup.get_text().replace(',', ''))

    urls = []
    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")

    has_more = scroll(1,browser,last_height)

    while len(urls) < num_posts:   

        url_element = browser.find_elements_by_css_selector('div.v1Nh3.kIKUG._bz0w')

        for url in url_element:
            url = url.get_attribute('innerHTML')
            soup = BeautifulSoup(url, 'html.parser')
            urls.append(soup.find('a')['href'])

        urls = list(dict.fromkeys(urls))
        print(len(urls))
        has_more = scroll(1,browser,last_height)
        
    print(username + " " + str(len(urls)))

    browser.close()

    return urls

In [10]:
def scroll(times,browser,last_height):
    SCROLL_PAUSE_TIME = 3

    scrolling = times
    while scrolling > 0:
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            return False
        last_height = new_height
        scrolling -= 1
    return True

In [11]:
def scrape_likescomments(urls):
    posts = []
    dates = []
    likes = []
    comment_count = []
    commenters = []
    comments = []
    
    for url in list(urls):
        url = 'http://instagram.com' + url
        print(url)
        page = urlopen(url).read()
        soup = BeautifulSoup(page, features="html.parser")

        string = soup.find("meta",  property="og:description")['content']
        string = string.split(" ")
        num_likes = string[0]
        num_comments = string[2]

        browser = create_webdriver()
        browser.get(url)
        
        date = browser.find_element_by_css_selector('time._1o9PC.Nzb55').get_attribute('datetime')
        print(date)

        hasLoadMore = True
        while hasLoadMore:
            time.sleep(2)
            try:
                browser.find_element_by_css_selector('button.dCJp8.afkep > span.glyphsSpriteCircle_add__outline__24__grey_9.u-__7').click()
            except:
                hasLoadMore = False

        texts = browser.find_elements_by_class_name('C4VMK')
       
        for comment in texts:
            posts.append(url)
            dates.append(date)
            likes.append(num_likes)
            comment_count.append(num_comments)
            try:
                comments.append(comment.text.split("\n")[1])
                commenters.append(comment.text.split("\n")[0])
            except:
                commenters.append(" ")
                comments.append(" ")

        browser.close()

    df = pd.DataFrame({'url': posts, 
                       'num_likes': likes, 
                       'date' : dates,
                       'num_comment': comment_count, 
                       'commenter': commenters, 
                       'comment': comments})
    return df
    

In [21]:
if not os.path.exists(os.getcwd()+"/data/LikesComments/"):
    os.makedirs(os.getcwd()+"/data/LikesComments/")
    
for username in open("usernames.txt", "r"):
    urls = get_posturls(username.strip())
    df = scrape_likescomments(urls)
    df['username'] = username.strip()
    
    display(df)
    
    df.to_csv(os.getcwd()+"/data/LikesComments/"+username.strip()+"-"+str(date.today())+'.txt',sep='\t',index = False,encoding='utf-8')


/Users/ellpeeaxe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


24
25
thewokesalaryman 25
http://instagram.com/p/B3nuS8blveP/
2019-10-15T01:37:51.000Z
http://instagram.com/p/B3ONIoLF2pZ/
2019-10-05T03:47:04.000Z
http://instagram.com/p/B2qNZtvl7Qy/
2019-09-21T04:16:44.000Z
http://instagram.com/p/B2fdmabFWhZ/
2019-09-17T00:06:38.000Z
http://instagram.com/p/B2IOTsXFKIl/
2019-09-07T23:30:29.000Z
http://instagram.com/p/B19_nJGlE_U/
2019-09-04T00:09:39.000Z
http://instagram.com/p/B1n9ww7l9SL/
2019-08-26T10:50:12.000Z
http://instagram.com/p/B1YBmNClAef/
2019-08-20T06:15:52.000Z
http://instagram.com/p/B1LWwcXF5-n/
2019-08-15T08:10:38.000Z
http://instagram.com/p/B1APpNAlY_i/
2019-08-11T00:36:50.000Z
http://instagram.com/p/B08eWSVFE0x/
2019-08-09T13:28:22.000Z
http://instagram.com/p/B0u-9ODFRvs/
2019-08-04T07:43:56.000Z
http://instagram.com/p/B0lZWrFBS9t/
2019-07-31T14:22:11.000Z
http://instagram.com/p/B0ZjyrbFE8n/
2019-07-27T00:02:30.000Z
http://instagram.com/p/B0JHsvVFIkO/
2019-07-20T14:49:11.000Z
http://instagram.com/p/Bz3U9fHF_d6/
2019-07-13T16:58:44.000

,comment,commenter,num_comment,num_likes,url,username
0,Fly 5 times to Bali every year? I’ve got an al...,thewokesalaryman,4,475,http://instagram.com/p/B3nuS8blveP/,thewokesalaryman
1,"Instead of flying, cycle? Insane mtf",junhonglife,4,475,http://instagram.com/p/B3nuS8blveP/,thewokesalaryman
2,Just nice for my Taiwan cycling trip next year...,hainanqueypneng,4,475,http://instagram.com/p/B3nuS8blveP/,thewokesalaryman
3,loved this!! super in depth hehe thank you! ✨✨,tehmoppity,4,475,http://instagram.com/p/B3nuS8blveP/,thewokesalaryman
4,Awesome!!!,bicarasepatu,4,475,http://instagram.com/p/B3nuS8blveP/,thewokesalaryman
5,Damn you @instagram for the 10 photo limit. Fo...,thewokesalaryman,44,"1,885",http://instagram.com/p/B3ONIoLF2pZ/,thewokesalaryman
6,water treaders rise up,cyan39miku,44,"1,885",http://instagram.com/p/B3ONIoLF2pZ/,thewokesalaryman
7,@zacjay,gabrielcjj,44,"1,885",http://instagram.com/p/B3ONIoLF2pZ/,thewokesalaryman
8,what if actually the mountain is stay afloat o...,myriad_color_hasan,44,"1,885",http://instagram.com/p/B3ONIoLF2pZ/,thewokesalaryman
9,Can anyone recommend a brand similar to @thewo...,nencorr,44,"1,885",http://instagram.com/p/B3ONIoLF2pZ/,thewokesalaryman


24
36
48
60
72
84
96
108
120
132
144
156
168
180
188
seedlysg 188
http://instagram.com/p/B3oA57El8j7/
2019-10-15T04:20:27.000Z
http://instagram.com/p/B2OgCJfFGjF/
2019-09-10T10:00:52.000Z
http://instagram.com/p/B07TsrWFvve/
2019-08-09T02:36:03.000Z
http://instagram.com/p/B00lfBlFvof/
2019-08-06T11:56:48.000Z
http://instagram.com/p/B0x_jiPhb__/
2019-08-05T11:46:53.000Z
http://instagram.com/p/B0qTqahloia/
2019-08-02T12:08:39.000Z
http://instagram.com/p/B0ijbCJFpdv/
2019-07-30T11:52:27.000Z
http://instagram.com/p/B0K5nOfluWL/
2019-07-21T07:24:34.000Z
http://instagram.com/p/B0IxO-oFJSH/
2019-07-20T11:32:53.000Z
http://instagram.com/p/Bz74WytFAYr/
2019-07-15T11:24:59.000Z
http://instagram.com/p/Bz2o1R4FRu_/
2019-07-13T10:33:08.000Z
http://instagram.com/p/BzvD6i-lmIl/
2019-07-10T11:55:52.000Z
http://instagram.com/p/BzsekYFFLnC/
2019-07-09T11:51:02.000Z
http://instagram.com/p/BzsSX7dlyJ-/
2019-07-09T10:04:29.000Z
http://instagram.com/p/Bzp5nS5lmle/
2019-07-08T11:49:39.000Z
http://instagram.co

ConnectionResetError: [Errno 54] Connection reset by peer

In [21]:
username = "seedlysg"
urls = get_posturls(username.strip())
df = scrape_likescomments(urls)
df['username'] = username.strip()

df.to_csv(os.getcwd()+"/data/LikesComments/"+username.strip()+"-"+str(date.today())+'.txt',sep='\t',index = False,encoding='utf-8')



/Users/ellpeeaxe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


24
36
48
60
72
84
96
108
120
132
144
156
168
180
188
seedlysg 188
http://instagram.com/p/B3oA57El8j7/
2019-10-15T04:20:27.000Z
http://instagram.com/p/B2OgCJfFGjF/
2019-09-10T10:00:52.000Z
http://instagram.com/p/B07TsrWFvve/
2019-08-09T02:36:03.000Z
http://instagram.com/p/B00lfBlFvof/
2019-08-06T11:56:48.000Z
http://instagram.com/p/B0x_jiPhb__/
2019-08-05T11:46:53.000Z
http://instagram.com/p/B0qTqahloia/
2019-08-02T12:08:39.000Z
http://instagram.com/p/B0ijbCJFpdv/
2019-07-30T11:52:27.000Z
http://instagram.com/p/B0K5nOfluWL/
2019-07-21T07:24:34.000Z
http://instagram.com/p/B0IxO-oFJSH/
2019-07-20T11:32:53.000Z
http://instagram.com/p/Bz74WytFAYr/
2019-07-15T11:24:59.000Z
http://instagram.com/p/Bz2o1R4FRu_/
2019-07-13T10:33:08.000Z
http://instagram.com/p/BzvD6i-lmIl/
2019-07-10T11:55:52.000Z
http://instagram.com/p/BzsekYFFLnC/
2019-07-09T11:51:02.000Z
http://instagram.com/p/BzsSX7dlyJ-/
2019-07-09T10:04:29.000Z
http://instagram.com/p/Bzp5nS5lmle/
2019-07-08T11:49:39.000Z
http://instagram.co

2018-06-07T07:34:39.000Z
http://instagram.com/p/BjpHte3nN8A/
2018-06-05T12:11:17.000Z
http://instagram.com/p/Bjb--6vHnHy/
2018-05-31T09:44:53.000Z
http://instagram.com/p/BjbTCTEnI8B/
2018-05-31T03:20:52.000Z
http://instagram.com/p/BjMbpwDn8RK/
2018-05-25T08:47:33.000Z
http://instagram.com/p/Biyy3PCn3a3/
2018-05-15T09:50:07.000Z
http://instagram.com/p/BiwBt84nyvx/
2018-05-14T08:02:12.000Z
http://instagram.com/p/BidhxWkn9YQ/
2018-05-07T03:36:43.000Z
http://instagram.com/p/BiOS871HGSS/
2018-05-01T05:38:37.000Z
http://instagram.com/p/BiLsp_sHFPx/
2018-04-30T05:25:30.000Z
http://instagram.com/p/BiIwO0THQhw/
2018-04-29T01:59:01.000Z
http://instagram.com/p/Bh6YDuyneY1/
2018-04-23T11:58:26.000Z
http://instagram.com/p/Bh4BBHlncGm/
2018-04-22T13:58:37.000Z
http://instagram.com/p/BhsnOUbnLXv/
2018-04-18T03:41:35.000Z
http://instagram.com/p/BhlMiKbHpfU/
2018-04-15T06:33:40.000Z
http://instagram.com/p/BhalDkVneMa/
2018-04-11T03:36:18.000Z
http://instagram.com/p/BhT7kHIjJWP/
2018-04-08T13:38:18.000Z

In [13]:
display(df)

,comment,commenter,date,num_comment,num_likes,url,username
0,Fly 5 times to Bali every year? I’ve got an al...,thewokesalaryman,2019-10-15T01:37:51.000Z,4,475,http://instagram.com/p/B3nuS8blveP/,thewokesalaryman
1,"Instead of flying, cycle? Insane mtf",junhonglife,2019-10-15T01:37:51.000Z,4,475,http://instagram.com/p/B3nuS8blveP/,thewokesalaryman
2,Just nice for my Taiwan cycling trip next year...,hainanqueypneng,2019-10-15T01:37:51.000Z,4,475,http://instagram.com/p/B3nuS8blveP/,thewokesalaryman
3,loved this!! super in depth hehe thank you! ✨✨,tehmoppity,2019-10-15T01:37:51.000Z,4,475,http://instagram.com/p/B3nuS8blveP/,thewokesalaryman
4,Awesome!!!,bicarasepatu,2019-10-15T01:37:51.000Z,4,475,http://instagram.com/p/B3nuS8blveP/,thewokesalaryman
5,Damn you @instagram for the 10 photo limit. Fo...,thewokesalaryman,2019-10-05T03:47:04.000Z,44,"1,889",http://instagram.com/p/B3ONIoLF2pZ/,thewokesalaryman
6,water treaders rise up,cyan39miku,2019-10-05T03:47:04.000Z,44,"1,889",http://instagram.com/p/B3ONIoLF2pZ/,thewokesalaryman
7,@zacjay,gabrielcjj,2019-10-05T03:47:04.000Z,44,"1,889",http://instagram.com/p/B3ONIoLF2pZ/,thewokesalaryman
8,what if actually the mountain is stay afloat o...,myriad_color_hasan,2019-10-05T03:47:04.000Z,44,"1,889",http://instagram.com/p/B3ONIoLF2pZ/,thewokesalaryman
9,Can anyone recommend a brand similar to @thewo...,nencorr,2019-10-05T03:47:04.000Z,44,"1,889",http://instagram.com/p/B3ONIoLF2pZ/,thewokesalaryman


,comment,commenter,date,num_comment,num_likes,url,username
0,Fly 5 times to Bali every year? I’ve got an al...,thewokesalaryman,2019-10-15T01:37:51.000Z,4,475,http://instagram.com/p/B3nuS8blveP/,thewokesalaryman
1,"Instead of flying, cycle? Insane mtf",junhonglife,2019-10-15T01:37:51.000Z,4,475,http://instagram.com/p/B3nuS8blveP/,thewokesalaryman
2,Just nice for my Taiwan cycling trip next year...,hainanqueypneng,2019-10-15T01:37:51.000Z,4,475,http://instagram.com/p/B3nuS8blveP/,thewokesalaryman
3,loved this!! super in depth hehe thank you! ✨✨,tehmoppity,2019-10-15T01:37:51.000Z,4,475,http://instagram.com/p/B3nuS8blveP/,thewokesalaryman
4,Awesome!!!,bicarasepatu,2019-10-15T01:37:51.000Z,4,475,http://instagram.com/p/B3nuS8blveP/,thewokesalaryman
5,Damn you @instagram for the 10 photo limit. Fo...,thewokesalaryman,2019-10-05T03:47:04.000Z,44,"1,889",http://instagram.com/p/B3ONIoLF2pZ/,thewokesalaryman
6,water treaders rise up,cyan39miku,2019-10-05T03:47:04.000Z,44,"1,889",http://instagram.com/p/B3ONIoLF2pZ/,thewokesalaryman
7,@zacjay,gabrielcjj,2019-10-05T03:47:04.000Z,44,"1,889",http://instagram.com/p/B3ONIoLF2pZ/,thewokesalaryman
8,what if actually the mountain is stay afloat o...,myriad_color_hasan,2019-10-05T03:47:04.000Z,44,"1,889",http://instagram.com/p/B3ONIoLF2pZ/,thewokesalaryman
9,Can anyone recommend a brand similar to @thewo...,nencorr,2019-10-05T03:47:04.000Z,44,"1,889",http://instagram.com/p/B3ONIoLF2pZ/,thewokesalaryman
